# Dealing with Few to No Labels

At the start of every project, the first question any data scientist asks? Is there any labelled data? Answer's mostly no or little bit. And on top of client's expectation that our fancy machine learning models should still perform well. One obvious apporach is annotate more data, but this is more expensive if each annotation has to be validated by a domain expert.

Fortunaley there are several methods that are suited for dealing with few to no labels. Like *Zero-shot* in GPT-3 where it perfoms over a diverse range of tasks with a few dozen examples.

In general, the best-performing method will depend upon task, data, how much of the data is labeled. 

*The below picture will guide us through the process of picking the most appropriate method*
![alt](../notes/images/9-dealing-with-few-to-no-labels/technqiques-to-deal-with-less-to-no-labelled-data.png)

Let's walk through this decision tree:

1. Is labeld data available?

Evan a handful of labeled of labeled samples can make a difference with regards to which method works best. If no labeld data is available then we can start with zero-shot learning which often sets a strong baseline to work from.

2. How many labels?

If we lots of labelled training data available then we can use the fine tuning approach used in [notebook 2](../notebooks/2-text-classification.ipynb)

3. Is there unlabeled data available?

If we have a handful of labeled samples it can help immensley if we have access to large amounts ot unlabeled data. If we have access to unlabeled data, we can use it to fine-tune the language model on the domain before training the classifier, or use more sophisticated methods like unsupervised data augmentation(UDA) or uncertainy-aware self-training(UST). If no unlabeled data is available, we can't annotate more data. In this case we can use few-shot learning or use the embeddings from a pretrained language model to perform lookups with a nearest neighbor search.

Int this notebook, we'll work our way through the decision tree by tackling a common problem facing many support teams that use issue trackers like Jira or Github to assist their users: tagging issues with metadata based on the issue's description. These tags define issue type,product causing the issue, responsible team. Automating the process will have an big impact on productivity and enables the support teams to focus on helping the users. In this notebook, we'll use issues associated with a populat open source project: Transformers. Let's now take a look at what information is available in these issues, what is the task and how to get the data.

> **Note:** The methods used in this notebooks will work well for text classification, but other techniques such as data augmentation may be necessary for dealing with more complex tasks like named entity recognition, question answering or summarization.

## Building a Github Issues Tagger

If we navgate to [issues tab](https://github.com/huggingface/transformers/issues) of transformers, we'll have each issues with title, description, set of tags or labels to chracterize the issue. The supervised learning task: given a title, description of an issue, predict one or more labels, this means we are dealing with multlable classification problem. 

*Single issue*
![alt](../notes/images/9-dealing-with-few-to-no-labels/singe-issue.png)

Now we've seen how the Github Issues look like, next let's see how to download and create our dataset.

To grab all repository's issues, we'll use the [GitHub REST API](https://docs.github.com/en/rest?apiVersion=2022-11-28) to poll the [Issues endpoint](https://docs.github.com/en/rest/issues#list-repository-issues). This endpoint returns a list of json objects, with each object contatining, issue, title, description, whether issue is open or close, owho opend it etc.

Since it takes a while to fetch all issues, we'll use the *github-issues-transformers-json* file. Along with `fetch_issues()` function to download them.

> **Note:** The GitHub REST API treats pull requests as issues, so our dataset contains both issues and pull requests. To keep things simple. we'' develop our classifier for both issue types, althoug in practice we can have two seperated classifiers to have more fine-grained control over the model's performance.

In [3]:
import time
import math
import requests
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

def fetch_issues(
    owner="huggingface",
    repo="transformers",
    num_issues=10_000,
    rate_limit=5_000,
):
    batch = []
    all_issues = []
    per_page = 100 # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}")
        batch.extend(issues)

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = [] # Flush batch for next time period
            print(f"Reached Github rate limit. Sleeping for one hour...")
            time.sleep(60 * 60 + 1)
    all_issues.extend(batch)
    df = pd.DataFrrame.from_records(all_issues)
    df.to_json(f"github-issues-{repo}.jsonl", orient="records", lines=True)

This function will download all the issues in batches to avoid exceeding GitHub's limit on number of requests per hour. Let's download the file.

### Preparing the data

In [4]:

import pandas as pd

dataset_url = "https://git.io/nlp-with-transformers"
df_issues = pd.read_json(dataset_url, lines=True)
print(f"DataFrame shape: {df_issues.shape}")